# Introduction to Genki Widgets

A small plotting library which provides a python interface via jupyter widgets to the frontend application we are developing.

In [1]:
import genki_widgets as gw

import os 
os.chdir('/Users/bjarnihaukurbjarnason/documents/genki-signals')

import genki_signals.signals as s
from genki_signals.buffers import DataBuffer
from genki_signals.data_sources import Sampler

In [2]:

import time
from threading import Thread

from genki_signals.data_sources import SineWave

sine_buffer = DataBuffer(max_size=400)
sine_wave = SineWave(frequency=1, amplitude=5, phase=0)
sine_source = Sampler(sources={'sine': sine_wave}, sample_rate=100)

def run_source(buffer, source):
    source.start()
    while True:
        buffer.extend(source.read())
        time.sleep(1 / 50)
        if not source.is_active:
            return
        
t1 = Thread(target=run_source, args=(sine_buffer, sine_source))
t1.start()

In [3]:
sine_buffer

DataBuffer(max_size=400, data=timestamp: (130,)
sine: (130,))

In [4]:
# create a simple line plot

line = gw.Line(("timestamp", 0), ("sine", 0), x_axis_visible=False, n_visible_points=300)
line

Line(n_visible_points=300, sig_x={'key': 'timestamp', 'idx': 0, 'name': ''}, sig_y=[{'key': 'sine', 'idx': 0, …

In [5]:
sine_buffer.add_callback(line.update)


In [6]:
# sine_source.stop(); t1.join()

In [7]:
from genki_signals.data_sources import MouseDataSource


mouse_buffer = DataBuffer(max_size=100)
mouse_source = Sampler({'mouse_position': MouseDataSource()}, sample_rate=100)

t2 = Thread(target=run_source, args=(mouse_buffer, mouse_source))
t2.start()

Each chart can display itself or it can be wrapped in a `gw.Dashboard` widget which enables us to have many charts in the same window.

In [8]:
sig_x = ("timestamp", 0)
sig_y = [
    ("mouse_position", 0, "x"),
    ("mouse_position", 1, "y"),
]

line = gw.Line(sig_x, sig_y, x_axis_visible=False)
bar = gw.Bar(sig_y)

dashboard = gw.Dashboard([line, bar])
dashboard

Dashboard(plot_widgets=[Line(sig_x={'key': 'timestamp', 'idx': 0, 'name': ''}, sig_y=[{'key': 'mouse_position'…

In [9]:
mouse_buffer.add_callback(dashboard.update)

In [10]:
line.x_axis_flipped = True

In [11]:
line

Line(sig_x={'key': 'timestamp', 'idx': 0, 'name': ''}, sig_y=[{'key': 'mouse_position', 'idx': 0, 'name': 'x'}…

In [12]:
mouse_buffer.add_callback(line.update)

This library can also handle an incredible throughput of data. A good example of which is the `gw.Spectrogram` widget.

In [13]:
line3 = gw.Line(sig_x, sig_y, x_axis_visible=False)
line3

Line(sig_x={'key': 'timestamp', 'idx': 0, 'name': ''}, sig_y=[{'key': 'mouse_position', 'idx': 0, 'name': 'x'}…

In [14]:
mouse_buffer.add_callback(line3.update)

In [15]:
from genki_signals.system import System
from genki_signals.data_sources import MicDataSource

audio_buffer = DataBuffer(400)
mic_source = MicDataSource()
derived = [
    s.FourierTransform('audio', name='spectro', window_size=2048, window_overlap=1024)
]

system = System(
    mic_source,
    derived
)

In [16]:
import time
from threading import Thread

def run_system():
    system.start()
    while True:
        audio_buffer.extend(system.read())
        time.sleep(1 / 25)
        if not system.running: 
            return
        
t3 = Thread(target=run_system)
t3.start()

In [17]:
audio_buffer

DataBuffer(max_size=400, data=audio: (400,)
spectro: (1025, 400))

Here we create the aforementioned spectrogram which contains `1025x500=512.500` datapoints!

In [18]:
spectro = gw.Spectrogram(
    ("spectro", 0),
    window_size = 2048,
    sampling_rate = mic_source.sample_rate,
    n_visible_windows = 500,
    colormap_max = 3,
)
spectro

Spectrogram(colormap_max=3.0, n_visible_windows=500, sig_x={'key': '', 'idx': 0, 'name': ''}, sig_y=[{'key': '…

In [19]:
audio_buffer.add_callback(spectro.update)


In [ ]:
1-1